In [7]:
!pip install -q keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.datasets import imdb
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [9]:
# Load the data
(num_words, maxlen) = 10000, 500  # Top 10,000 most frequent words, truncating reviews to 500 words
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

# Pad sequences to ensure uniform length
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [10]:
# Function to create the BiRNN Model
def create_birnn_model(hp):
    embedding_dim = hp.Int('embedding_dim', min_value=64, max_value=128, step=64)
    rnn_units = hp.Int('rnn_units', min_value=64, max_value=256, step=64)
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.3, step=0.1)

    model = models.Sequential([
        layers.Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=maxlen),
        layers.Bidirectional(layers.LSTM(rnn_units, dropout=dropout_rate)),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Use Hyperband for hyperparameter tuning
tuner = kt.Hyperband(
    create_birnn_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='imdb_birnn_tuning'
)

In [ ]:
# Run hyperparameter search
tuner.search(X_train, y_train, epochs=3, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
y_pred_best = best_model.predict(X_test)
y_pred_binary_best = (y_pred_best > 0.5).astype(int)

accuracy_best = accuracy_score(y_test, y_pred_binary_best)
precision_best = precision_score(y_test, y_pred_binary_best)
recall_best = recall_score(y_test, y_pred_binary_best)
f1_best = f1_score(y_test, y_pred_binary_best)
roc_auc_best = roc_auc_score(y_test, y_pred_best)

print("\nBest Model Evaluation Metrics:")
print(f'Accuracy: {accuracy_best:.4f}')
print(f'Precision: {precision_best:.4f}')
print(f'Recall: {recall_best:.4f}')
print(f'F1 Score: {f1_best:.4f}')
print(f'ROC AUC: {roc_auc_best:.4f}')

Trial 1 Complete [00h 12m 18s]
val_accuracy: 0.8669999837875366

Best val_accuracy So Far: 0.8669999837875366
Total elapsed time: 00h 12m 18s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
64                |64                |embedding_dim
192               |64                |rnn_units
0.2               |0.2               |dropout_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 1430s 2s/step - accuracy: 0.6579 - loss: 0.6076 - val_accuracy: 0.8334 - val_loss: 0.4020
Epoch 2/2
559/625 ━━━━━━━━━━━━━━━━━━━━ 2:20 2s/step - accuracy: 0.8613 - loss: 0.3368